# Tutorial 1: Running global CTSM simulations 

This tutorial is an introduction to running global scale simulations of the Community Terrestrial Systems Model (CTSM).  It will guide you through running a simulation and provides example visualization of simulation results. Once you complete the global simulation in this tutorial, you can use the `Global_Visualization_Tutorial` to explore the model data further.

In the previous `NEON_Simulation_Tutorial` example many steps to run CLM were condensed into a single function. Here you'll how to create your own case, setup, build, and run the simulation. *These are the basic steps reqired to run CLM and modify the code to conduct model experiments*
<br><br>

### A few notes about the model:
- There are several configuration options of CTSM, and throughout this tutorial we will use the Community Land Model (CLM) configuration which is the climate mode of CTSM. Throughout the rest of this tutorial, we refer to the model as CLM and will use version 5.1 with satelity phenology (**CLM5.1-SP**).

- **Satelite phenology (or SP)** simulations prescribe he distribution of vegetation and its leaf area index (LAI). These SP simulations useful for understanding water and energy fluxes that are simulated by the model, as well as gross primary productivity (GPP). SP simulations focus on the *biogeophysics* of the model.  Accordingly, there is no *biogeochemistry* (i.e., carbon fluxes) in SP simulations downstream of GPP calculations. This simplification removes potential feedbacks or biases that may occur from plant allocation, autotrophic respiration, and turnover on simulated LAI. SP simulations can be conducted with CLM5.1-SP, or with CLM_FATES-SP.

- **FATES** - the Functionally Assembled Terrestrial Ecosystem Simulator - is an external module which can run within a given “Host Land Model”, here we are using CLM.  This examples uses the satelite phenology scheme in FATES, one of the reduced complexity modes of FATES that helps to isolate representation of canopy processes like radiative transfer and photosynthesis. The full FATES model will allow researchers to investigate the influence of plant demographics and disturbance on ecosystem processes in a changing world with greater ecological realism than is possible with current vegetation model in CLM. FATES compsets are available in CLM.  
<br><br>

### Additional information:
Additional information about CTSM and CLM is available [on the website](https://www.cesm.ucar.edu/models/cesm2/land/), including [technical documentation](https://escomp.github.io/ctsm-docs/versions/master/html/tech_note/index.html), a [user's guide](https://escomp.github.io/ctsm-docs/versions/master/html/users_guide/index.html), and a [quickstart guide](https://escomp.github.io/CESM/release-cesm2/quickstart.html#create-a-case) for running various model configurations beyond what is covered in this tutorial.

Additional information about FATES is available [on the FATES github site](https://github.com/NGEET/fates). TODO ADD additional documentation here.
<br><br>
### Questions about this tutorial? 
- Please post them on the [CTSM forum in the CESM Bulletin Board](https://bb.cgd.ucar.edu/cesm/forums/ctsm-clm-mosart-rtm.134/). Note that this resource will require you to register and log in so that you can be notified of responses to your inquiries. 
- You can also file issues on the [NCAR CTSM-Tutorial GitHub repository](https://github.com/NCAR/CTSM-Tutorial-2022).
***

## In this tutorial

The tutorial has several components. Below you will find steps to: 
1. Run an out-of-the-box CLM simulation using CLM_FATES-SP
2. Locate model history files
3. Basic visualization of model history files

***
**This tutorial uses a Jupyter Notebook.** A Jupyter Notebook is an interactive computing environment that enables the creation and sharing of documents that contain discrete cells of text or documentation and executable code, including plots. It allows users to access, run, and edit the code in a browser. To interact with this notebook:

- Execute or "run" cells of executable code (cells denoted with '[ ]:') using the play button in the menu at the top (see below image)

- The results of running code, such as plots, will appear below that cell

- Each step must be followed in order, otherwise you may encounter errors

![run cell](https://problemsolvingwithpython.com/02-Jupyter-Notebooks/images/run_cell.png)

For more information on Jupyter notebooks please see the [Jupyter Notebook Quick Start Guide](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html). 
***

## Note This tutorial assumes that you've done your homework! 

If you haven't downloaded CTSM from the github repository you need to go back to the `NEON_Simulation_Tutorial` and do this first.
***

<h2> 1. Set up and run a simulation</h2>

**CTSM can be run in 4 simple steps.** 

<h4>1.1 create a new case </h4>

- *This step sets up a new simulation. It is the most complicated of these four steps because it involves making choices to set up the model configuration*

#### 1.2 invoke case.setup
- *This step configures the model so that it can compile*

#### 1.3 build the executable
- *This step compiles the model*

#### 1.4 submit your run to the batch queue
- *This step submits the model simulation to the cloud*

***
<h2> 1.1 create a new case</h2>

In [1]:
# First, a one-time step to create a directory to store your experiment cases:
# TODO, do I need the ! at the start of lines in the notebook?
!mkdir ~/clm_tutorial_cases

mkdir: cannot create directory ‘/glade/u/home/wwieder/clm_tutorial_cases’: File exists


In [2]:
# Then, navigate to the scripts directory in the source code directory:
# TODO ADJUST FOR DIRECTORY STRUCTURE IN THE CLOUD
!cd /glade/u/home/$USER/ctsm51dev090/cime/scripts

/usr/bin/sh: line 0: cd: /glade/u/home/wwieder/ctsm51dev090/cime/scripts: No such file or directory


In [3]:
# Create a new case
./create_newcase --case ~/clm_tutorial_cases/I2000_CTSM_FATES-SP --res f19_g17 --compset I2000Clm51Fates

SyntaxError: invalid syntax (<ipython-input-3-e1b4457da5c1>, line 2)

You should see an error message when you try to create this case.  

Repeat the line above with the additional command suggested

***
## TODO 
shoud we just add the --run-unsupported from the start?
./create_newcase --case ~/clm_tutorial_cases/I2000_CTSM_FATES-SP --res f19_g17 --compset I2000Clm51Fates --run-unsupported

## TODO
what's the best way to include information about parts needed for `./create_newcase`?

[See slides 26-29](https://www.cgd.ucar.edu/events/2019/ctsm/files/practical1-lombardozzi.pdf)

<br><br>
The model is done running when you see the following line: 


> Submit job case.st_archive 


### Congratulations! You've created and run CLM for the NEON tower you selected.
Follow the steps below to see a few example plots from simulated data. 

**Note:** If your model simulation did not complete, there will likely be an error message with more information. If you do not understand the error message, please post your question on the [CTSM forum in the CESM Bulletin Board](https://bb.cgd.ucar.edu/cesm/forums/ctsm-clm-mosart-rtm.134/).


***
<h4> Optional </h4>

If you would like to see additional and more advanced options for running NEON tower site simulations, many are listed in the python script you just executed. You can use the `--help` option to see more. <p>

*Executing the below cell is optional and will print options available options in the `run_neon` python script*:

In [ ]:
run_neon --help

------------

#### Below are a few additional tips for running simulations: 

 __Tip:__ You can run multiple neon sites at once by choosing multiple neon sites in the command line argument. For example:
>
>```
>run_neon.py --neon-sites ABBY BART BLAN
>```

<br/>

 __Tip:__ Use the `--overwrite` option to overwrite existing case directories.
>
>If you run a simulation for a NEON site that you have previously simulated, you will see the following error:
>```
>Case already exists in /home/user/CLM-NEON/ABBY.transient, not overwritting.
>```
>The overwrite option (e.g., adding `--overwrite` to the end of the `run_neon` command) can be used to avoid this error. 

<br/>

 __Tip:__ You can run the above `run_neon` command in your terminal shell. You will need to specify the NEON site and the output-root path by replacing `<site>` and `<path>` below with the appropriate values:

>```
>run_neon.py --neon-sites <site> --output-root <path>
>```

<br/>
    
 __Tip:__ The container will not overwrite existing files. If there are newer versions of a tutorial or input data you want to download, you must delete the existing tutorials or data first. You can do this by moving the files from the `Simulations` folder on your Desktop into the trash.
    
<br/>
    
**Note:** More information about setting up and modifying simulations, including changing model code, will be available in an upcoming tutorial. 


________
<h2> 2. Explore CTSM model data </h2>

*When your simulation completes, this step guides you through exploring the data.
There are countless ways of analyzing and processing model data. 
The below steps step through where to find model data and creates a plot to visualize some model data.*
***

<h3> 2.1 Locate model data </h3>

When a simulation completes, the data are stored in the `archive` directory. In this directory you will find files that include data for every day of the simulation, as well as files that average model variables monthly. <p>

*Run the code below to see a subset of the files listed.*

In [ ]:
!ls $HOME/archive/$site.transient/lnd/hist/*2018*.nc |head -20


Notice that each line includes the location of the file (`/home/user/archive/{simulation_name}/lnd/hist/`) and file name. The file names are automatically generated and are composed of:
* the simulation name, which includes:
    * the NEON site
    * the type of simulation 
        * The simulation you ran is "transient". This means the model was initialized and ran for the full length of available data. The initial conditions files for the transient tower simulations were created by cycling over 2018-2019 tower meteorological data. 
* the date of simulated data

The files are saved in netcdf format (denoted with the `.nc` file extension), a file format commonly used for storing large, multi-dimensional scientific variables.
Netcdf files are platform independent and self-describing; each file includes metadata that describes the data, including: **variables**, **dimensions**, and **attributes**. You can explore the files more in the NEON_Simple_Visualization_Tutorial

The NEON tower simulations generate two types of files:
* `*h0*`: Variables that are averaged monthly. One file is available for every month of the simulation. These files include hundreds of variables.
* `*h1*`: Variables that are recorded every 30 minutes. Values are aggregated into one file for each day of the simulation. Each file includes 48 data points for selected variables.

*Note that you can also find the model data in the 'Simulation' folder on your desktop, which was created as part of this tutorial.*
****


<h3>2.2 Visualize the output </h3>

Below we create a plot of simulated soil temperature using a predefined function, `plot_soil_profile_timeseries`.


The `plot_soil_profile_timeseries` function points to the simulated data files in your `archive` directory, extracts a variable (soil temperature, `TSOI`, in this case), and plots the soil profile. The y-axis shows soil depth and the x-axis shows time. Note that time is currently set for the year 2018. You can change the `year` variable below to plot different years from 2018 through 2020.

In [ ]:
from neon_utils import plot_soil_profile_timeseries
sim_path = "/home/user/archive/"+neon_site+".transient/lnd/hist/"
case_name = neon_site+".transient.clm2"
year = 2018

plot_soil_profile_timeseries(sim_path, neon_site, case_name, 'TSOI', year)

You might also be interested in the soil moisture at this site. The `plot_soil_profile_timeseries` function can also plot soil moisture (`H2OSOI`). Run the below cell to see the soil moisture profile for 2018. You can change the `year` variable below to plot different years from 2018 through 2020.

In [ ]:
sim_path = "/home/user/archive/"+neon_site+".transient/lnd/hist/"
case_name = neon_site+".transient.clm2"
year = 2018

plot_soil_profile_timeseries(sim_path, neon_site, case_name, 'H2OSOI', year)

### Congratulations! 
You have now completed a NEON tower simulation, located the files, and made a plot from the simulation data.

**Additional examples of visualizations and evaluation of your NEON simulation against NEON observations are available in the next tutorial, `NEON_Visualization_Tutorial`.**